Adapted from `rlcard/examples/leduc_holdem_dqn_pytorch.py`

In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath('./rlcard'))

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
import torch
import rlcard
from DQNAgent import DQNAgent
from rlcard.agents import RandomAgent, CFRAgent
from rlcard.utils import set_global_seed, tournament
from rlcard.utils import Logger

In [4]:
# Make environment
env = rlcard.make('leduc-holdem', config={'seed': 0})
eval_env = rlcard.make('leduc-holdem', config={'seed': 0})

In [5]:
# Set the iterations numbers and how frequently we evaluate the performance
evaluate_every = 10_000
evaluate_num = 10_000
episode_num = 1_000_000

In [6]:
# The paths for saving the logs and learning curves
log_dir = './experiments/leduc_holdem_dqn_result_million/'

In [7]:
# Set a global seed
set_global_seed(0)

In [8]:
agent = DQNAgent(
    env.action_num,
    env.state_shape[0],
    memory=2_000_000,
    lr=0.1,
    batch_size=128,
    update_every=128,
    eps_start=0.12,
    target_update=300,
    hidden_neurons=[64]
)

cfr_agent = CFRAgent(eval_env, model_path='./models/leduc_holdem_cfr_million')
cfr_agent.load()

random_agent = RandomAgent(action_num=eval_env.action_num)

env.set_agents([agent, agent])

eval_env.set_agents([agent, cfr_agent])

In [9]:
# Init a Logger to plot the learning curve
logger = Logger(log_dir)

In [10]:
for episode in range(episode_num):
    # Generate data from the environment
    trajectories, _ = env.run(is_training=True)

    # Feed transitions into agent memory, and train the agent
    for ts in trajectories[0]:
        agent.train(ts)

    # Evaluate the performance. Play with random agents.
    if episode % evaluate_every == 0:
        logger.log_performance(agent.weight_updates,
                               tournament(eval_env, evaluate_num)[0])


----------------------------------------
  timestep     |  0
  reward       |  -0.49285
----------------------------------------

----------------------------------------
  timestep     |  189
  reward       |  -0.33145
----------------------------------------

----------------------------------------
  timestep     |  383
  reward       |  -0.08425
----------------------------------------

----------------------------------------
  timestep     |  581
  reward       |  -0.15775
----------------------------------------

----------------------------------------
  timestep     |  765
  reward       |  -0.019
----------------------------------------

----------------------------------------
  timestep     |  953
  reward       |  -0.05745
----------------------------------------

----------------------------------------
  timestep     |  1136
  reward       |  -0.1202
----------------------------------------

----------------------------------------
  timestep     |  1310
  reward       


----------------------------------------
  timestep     |  10055
  reward       |  0.00655
----------------------------------------

----------------------------------------
  timestep     |  10180
  reward       |  0.0225
----------------------------------------

----------------------------------------
  timestep     |  10311
  reward       |  -0.053
----------------------------------------

----------------------------------------
  timestep     |  10454
  reward       |  0.04115
----------------------------------------

----------------------------------------
  timestep     |  10582
  reward       |  -0.01245
----------------------------------------

----------------------------------------
  timestep     |  10711
  reward       |  0.00115
----------------------------------------

----------------------------------------
  timestep     |  10835
  reward       |  -0.01185
----------------------------------------

----------------------------------------
  timestep     |  10946
  r

In [11]:
# Close files in the logger
logger.close_files()

In [12]:
# Plot the learning curve
logger.plot('DQN')

./experiments/leduc_holdem_dqn_result_million/performance.csv


In [14]:
# Save model
agent.save('./models/leduc_holdem_dqn_million/')

In [15]:
eval_env_dqn_dqn = rlcard.make('leduc-holdem', config={'seed': 1})
eval_env_dqn_dqn.set_agents([agent, agent])
payoffs = tournament(eval_env_dqn_dqn, 10000)
print(payoffs)

[-0.0034, 0.0034]
